In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install tensorflow==2.4.0
!pip install tensorflow==2.5.0
!pip install keras==2.4.0
#!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
sys.path.append("drive/MyDrive/DeSpaWN-main")
# Import Data
import scipy.io as sio
#from DataHandler import DataHandler
#from ModelHandler import ModelHandler
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
#from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight


configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/Scattering_Novo/dataset_original/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FEATURES_FILE_NAME": "features.mat",
    "OUTPUT_CLASSIFICATION_MODELS_PATH": "drive/MyDrive/DeSpaWN-main/classification/without_data_augmentation/",
    "REQUIREMENTS_PATH": "drive/MyDrive/DeSpaWN-main/classification/",
    "N_EPOCHS_TRAINING": 500,
    "FEAT_NORMALIZATION": True,
    "TRAINING_FLAG": 1,
    "PERCENTUAL": 1.0,
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None 
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 

#folderPath = configs["TESTS_FOLDER"] + 'Hybrid_P' + str(int(configs["PERCENTUAL"]*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
folderDataPath = configs["FOLDER_DATA_PATH"]

trainSize = configs["TRAIN_SIZE"]

dict_data = pickle.load(open(folderDataPath + "data.p", "rb")) # Load data

#modelHandler = ModelHandler(configs)

features_folder_name = configs["FOLDER_PATH"] + "RAW_Features" 



In [ ]:
training_flag=configs["TRAINING_FLAG"]

## Load Features

In [ ]:
import scipy.io as sio
import pickle


In [ ]:
full_training_set_path_G = configs["FOLDER_DATA_PATH"] + "combined_features_training_G.mat"
full_training_set_path_Gint = configs["FOLDER_DATA_PATH"] + "combined_features_training_Gint.mat"
full_training_set_path_H = configs["FOLDER_DATA_PATH"] + "combined_features_training_H.mat"

full_testing_set_path_G = configs["FOLDER_DATA_PATH"] + "combined_features_testing_G.mat"
full_testing_set_path_Gint = configs["FOLDER_DATA_PATH"] + "combined_features_testing_Gint.mat"
full_testing_set_path_H = configs["FOLDER_DATA_PATH"] + "combined_features_testing_H.mat"


training_features_folder_name = features_folder_name 
testing_features_folder_name = features_folder_name 

### Merging the Training Features

In [ ]:
'''
if os.path.exists(full_training_set_path_G):
  os.remove(full_training_set_path_G)

if os.path.exists(full_training_set_path_H):
  os.remove(full_training_set_path_H)
'''

if not os.path.exists(full_training_set_path_G):
    print(full_training_set_path_G)
    number_of_training_files= 76
    try: 
      for cont in range(1,number_of_training_files+1): # Number of training files

        path = "G_train_RAW_" + str(cont) + ".pkl"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)


              try:
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                N = len(imported)
                
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                  
                imported2 = np.zeros([N,10]) # 100 lines, one per example. 10 collumns, one per layer

                for n in range(N): # for each sample
                  for k in range(10): # for each level
                    imported2[n,k] = np.mean( imported[n][k].reshape([imported[n][k].shape[1],]) )
                    
 
                if cont==1:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")
    
    sio.savemat(full_training_set_path_G, {'arr': feats})
    print("Combined and saved!")
    del feats, imported2

if not os.path.exists(full_training_set_path_H):
    #feats= np.zeros([0,14])
    print(full_training_set_path_H)
    number_of_training_files= 76
    try: 
      for cont in range(1,number_of_training_files+1): # Number of training files
      #for cont in range(1): # Number of training files

        path = "H_train_RAW_" + str(cont) + ".pkl"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              #print(csv_path)
              #print("Extracting from " + path)
              try:
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                N = len(imported)
                
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                  
                imported2 = np.zeros([N,10]) # 100 lines, one per example. 10 collumns, one per layer

                for n in range(N): # for each sample
                  for k in range(10): # for each level
                    imported2[n,k] = np.mean( imported[n][k].reshape([imported[n][k].shape[1],]) )
                    
                #imported = sio.loadmat(csv_path)
                #imported2 = imported['arr']

                if cont==1:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")
    
    sio.savemat(full_training_set_path_H, {'arr': feats})
    print("Combined and saved!")
    del feats, imported2

   # print(imported.shape)

    #del feats, imported2


In [ ]:
X_train = sio.loadmat(full_training_set_path_H)
X_train = X_train['arr']
y_train = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_train_.mat")
y_train = y_train['arr']


In [ ]:
print(X_train.shape)
print(y_train.shape)

(7575, 10)
(7575, 5, 26)


### Merging the testing features

In [ ]:
'''
if os.path.exists(full_testing_set_path_G):
  os.remove(full_testing_set_path_G)

if os.path.exists(full_testing_set_path_H):
  os.remove(full_testing_set_path_H)
'''

if not os.path.exists(full_testing_set_path_G):
    #feats= np.zeros([0,14])
    print(full_testing_set_path_G)
    number_of_testing_files= 9
    try: 
      for cont in range(1,number_of_testing_files+1): # Number of training files
      #for cont in range(1): # Number of training files

        path = "G_test_RAW_" + str(cont) + ".pkl"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              #print(csv_path)
              #print("Extracting from " + path)
              try:
                
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                N = len(imported)
  
                imported2 = np.zeros([N,10]) # 100 lines, one per example. 10 collumns, one per layer

                for n in range(N): # for each sample
                  for k in range(10): # for each level
                    imported2[n,k] = np.mean( imported[n][k].reshape([imported[n][k].shape[1],]) )
                    
                #imported = sio.loadmat(csv_path)
                #imported2 = imported['arr']

                if cont==1:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")
    
    sio.savemat(full_testing_set_path_G, {'arr': feats})
    print("Combined and saved!")
    del feats, imported2

if not os.path.exists(full_testing_set_path_H):
    #feats= np.zeros([0,14])
    print(full_testing_set_path_H)
    number_of_testing_files= 9
    try: 
      for cont in range(1,number_of_testing_files+1): # Number of training files
      #for cont in range(1): # Number of training files

        path = "H_test_RAW_" + str(cont) + ".pkl"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              #print(csv_path)
              #print("Extracting from " + path)
              try:

                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                N = len(imported)
                
                with open(csv_path, 'rb') as f:
                  imported = pickle.load(f) # List with 10 levels and for each of one of the 100 samples per pack
                  
                imported2 = np.zeros([N,10]) # 100 lines, one per example. 10 collumns, one per layer

                for n in range(N): # for each sample
                  for k in range(10): # for each level
                    imported2[n,k] = np.mean( imported[n][k].reshape([imported[n][k].shape[1],]) )
                    
                #imported = sio.loadmat(csv_path)
                #imported2 = imported['arr']

                if cont==1:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")
    
    sio.savemat(full_testing_set_path_H, {'arr': feats})
    print("Combined and saved!")
    del feats, imported2

   # print(imported.shape)

    #del feats, imported2


10

In [ ]:
print(full_testing_set_path_H)

X_test  = sio.loadmat(full_testing_set_path_H)
X_test = X_test['arr']
y_test  = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_test_.mat")
y_test = y_test['arr']


drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/combined_features_testing_H.mat


In [ ]:
print("Shape of Classification Testing Features: ", X_test.shape)
print("Shape of Classification Testing Labels: ", y_test.shape)


Shape of Classification Testing Features:  (841, 10)
Shape of Classification Testing Labels:  (841, 5, 26)


## Features Normalization

In [ ]:
if configs["FEAT_NORMALIZATION"]:
  from sklearn.preprocessing import MultiLabelBinarizer
  from sklearn.preprocessing import MaxAbsScaler 
  transformer = MaxAbsScaler().fit(X_test)
  x_test_norm = transformer.transform(X_test)
  transformer = MaxAbsScaler().fit(X_train)
  X_train_norm = transformer.transform(X_train)

  print(x_test_norm.shape)
  print(X_train_norm.shape)

  X_test = x_test_norm
  X_train = X_train_norm

  del x_test_norm, X_train_norm

(841, 10)
(7575, 10)


## Training

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
#from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss

#from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import train_test_split



In [ ]:
seed=1
models = [
            'ADB',
            'GBC',
            'RFC',
            'KNC',
            'SVC',
            'logisticRegression'
         ]
clfs = [
        AdaBoostClassifier(random_state=seed),
        GradientBoostingClassifier(random_state=seed),
        RandomForestClassifier(random_state=seed,n_jobs=-1),
        KNeighborsClassifier(n_jobs=-1),
        SVC(random_state=seed,probability=True),
        LogisticRegression(solver='newton-cg', multi_class='multinomial')
        ]

params = {
            models[0]:{'learning_rate':[0.01], 'n_estimators':[150]},
            models[1]:{'learning_rate':[0.01],'n_estimators':[100], 'max_depth':[3],
                       'min_samples_split':[2],'min_samples_leaf': [2]},
            models[2]:{'n_estimators':[100], 'criterion':['gini'],'min_samples_split':[2],
                      'min_samples_leaf': [4]},
            models[3]:{'n_neighbors':[5], 'weights':['distance'],'leaf_size':[15]},
            models[4]: {'C':[100], 'tol': [0.005],
                       'kernel':['sigmoid']},
            models[5]: {'C':[2000], 'tol': [0.0001]}
         }

In [ ]:
Y_train = (np.sum(y_train,axis=1)>0)
Y_train2 = np.zeros([Y_train.shape[0],Y_train.shape[1]])

for k in range(Y_train.shape[0]): # for each line
  for kk in range(Y_train.shape[1]): # for each load
    if Y_train[k,kk]: # If true
      Y_train2[k,kk] = 1

Y_train = Y_train2
del Y_train2

print(Y_train[2019])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.1, random_state=0)

for train_index, test_index in sss.split(X_train, Y_train):
    n_x_train, n_x_val = X_train[train_index], X_train[test_index]
    n_y_train, n_y_val = Y_train[train_index], Y_train[test_index]

n_x_test = X_test
y_test = 0
test_scores = []

In [ ]:
n_x_val.shape

(758, 10)

In [ ]:
print(n_y_train.shape)



(6817, 26)


In [ ]:
if not os.path.exists(configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + 'individual_traditional_cls.pkl'):
  classifiers = []
  Classifiers = {}
  col=0
  for col in range(26): # training a classifieer for each load
        for name, estimator in zip(models,clfs):
            print(name)
            clf = GridSearchCV(estimator, params[name], scoring='neg_mean_squared_error', refit='True', n_jobs=-1, cv=5)
            clf.fit(n_x_train, n_y_train[:,col])

            #print("best params: " + str(clf.best_params_))
            #print("best scores: " + str(clf.best_score_))
            estimates = clf.predict_proba(n_x_test)
            y_test+=estimates
            acc = accuracy_score(n_y_val[:,col], clf.predict(n_x_val))
            #print("Accuracy: {:.4%}".format(acc))
            classifiers.append(clf)
            test_scores.append((acc,clf.best_score_))
        Classifiers[col] = classifiers
        classifiers = []
  
  #import pickle

  with open(configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + 'individual_traditional_cls.pkl', 'wb') as handle:
        pickle.dump(Classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
  
  with open(configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + 'individual_traditional_cls.pkl', 'rb') as f:
        Classifiers = pickle.load(f)



In [ ]:
len(Classifiers)

26

In [ ]:
y_test  = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_test_.mat")
y_test = y_test['arr']

In [ ]:
y_test.shape

(841, 5, 26)

# Predictions and Evaluation

In [ ]:
# Test labels 
Y_test = (np.sum(y_test,axis=1)>0)
Y_test2 = np.zeros([Y_test.shape[0],Y_test.shape[1]])

for k in range(Y_test.shape[0]): # for each line
  for kk in range(Y_test.shape[1]): # for each load
    if Y_test[k,kk]: # If true
      Y_test2[k,kk] = 1

Y_test = Y_test2
del Y_test2

print(Y_test.shape)

(841, 26)


## Binary predictions for each classifier

In [ ]:
Predictions = np.zeros((X_test.shape[0],26,len(Classifiers[0])))
Predictions.shape

(841, 26, 6)

In [ ]:
for load in range(26): # for each load
  for clf in range(len(Classifiers[load])):
    Predictions[:,load,clf] = Classifiers[load][clf].predict(X_test) 


In [ ]:
from sklearn.metrics import f1_score
F1_score = []

for cls in range(len(Classifiers[0])):
  y_pred = Predictions[:,:,cls]

  F1_score.append(f1_score(Y_test, y_pred, average='weighted'))

print(F1_score)

[0.0, 0.013314434845212384, 0.1304537651248588, 0.24411183490057223, 0.14635515041390457, 0.0018555658404470498]


## Probabilistic accumulated predictions

In [ ]:
predictions = 0
Predictions2 = []
for load in range(26): # for each load
  for clf in range(len(Classifiers[load])):
    predictions += Classifiers[load][clf].predict_proba(X_test) 
  Predictions2.append(predictions)
  predictions=0

Predictions2 = np.array(Predictions2)
proba_predictions = 1 / (1 + np.exp(-np.array(Predictions2)))


In [ ]:
proba_predictions[:,:,1].shape

(26, 841)

In [ ]:
#print(proba_predictions[16][800])

#print(Y_test[800][16])

threshold = 0.75

binary_predictions = proba_predictions[:,:,1] >= threshold
binary_predictions = binary_predictions.astype(int)

binary_predictions = binary_predictions.reshape([binary_predictions.shape[1], binary_predictions.shape[0]])
print(binary_predictions.shape)


(841, 26)


In [ ]:
f1 = f1_score(Y_test, binary_predictions, average='macro')
print(f1)

0.0913649131001427


## Saving Predictions

In [ ]:
predictions_path = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "Predictions_LW_Traditional/"

try:
   os.mkdir(predictions_path)
except:
   pass


sio.savemat(predictions_path + "Predictions_LW_Traditional" + ".mat", {'Predictions': Predictions})
sio.savemat(predictions_path + "Predictions_LW_Traditional_probab" + ".mat", {'Predictions': Predictions2})

In [ ]:
Predictions = sio.loadmat(predictions_path + "Predictions_LW_Traditional" + ".mat")
Predictions = Predictions["Predictions"]


Predictions2 = sio.loadmat(predictions_path + "Predictions_LW_Traditional_probab" + ".mat")
Predictions2 = Predictions2["Predictions"]